<a href="https://colab.research.google.com/github/AnthoMant/HACKATHON_study-on-space-missions/blob/main/Etude%20sur%20les%20mission%20spatiale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#data viz
import matplotlib.pyplot as plt
import seaborn as sns

#importing plotting libraries
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

#preprocess
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

#importing modeling libraries
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

#models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score, precision_score,recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#autres:
import time
import math as m
from scipy.spatial import distance
from google.colab import drive
from datetime import datetime
import re

In [ ]:
!pip install --upgrade plotly

     |████████████████████████████████| 13.2MB 271kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


# Importation de la base de données

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
space_cor=pd.read_csv('/content/drive/MyDrive/Les évadés/CSV/Space_Corrected.csv')

# Exploration de la base de données

In [ ]:
space_cor.head(30)

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success
5,5,5,CASC,"LC-9, Taiyuan Satellite Launch Center, China","Sat Jul 25, 2020 03:13 UTC","Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success
6,6,6,Roscosmos,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Thu Jul 23, 2020 14:26 UTC",Soyuz 2.1a | Progress MS-15,StatusActive,48.5,Success
7,7,7,CASC,"LC-101, Wenchang Satellite Launch Center, China","Thu Jul 23, 2020 04:41 UTC",Long March 5 | Tianwen-1,StatusActive,NaN,Success
8,8,8,SpaceX,"SLC-40, Cape Canaveral AFS, Florida, USA","Mon Jul 20, 2020 21:30 UTC",Falcon 9 Block 5 | ANASIS-II,StatusActive,50.0,Success
9,9,9,JAXA,"LA-Y1, Tanegashima Space Center, Japan","Sun Jul 19, 2020 21:58 UTC",H-IIA 202 | Hope Mars Mission,StatusActive,90.0,Success


In [ ]:
space_cor.drop(["Unnamed: 0" ,	"Unnamed: 0.1"], axis=1, inplace=True)

##Récuperation du mois et de l'année

In [ ]:
space_cor['Datum'] = pd.to_datetime(space_cor['Datum'])
space_cor['Year'] = space_cor['Datum'].apply(lambda datetime: datetime.year)
space_cor['Month'] = space_cor['Datum'].apply(lambda datetime: datetime.month)

In [ ]:
space_cor.drop("Datum",axis=1,inplace=True)

##Nettoyage préliminaire de la base de données

In [ ]:
#function to extract the name of the country from the location
def extract_country_name(location):
    country = location.split(',')[-1]
    country = country.strip()
    return country

#dictionary to help in mapping to get consistent and correct Country Names
countries_dict = {
    'Russia' : 'Russian Federation',
    'New Mexico' : 'USA',
    "Yellow Sea": 'China',
    "Shahrud Missile Test Site": "Iran",
    "Pacific Missile Range Facility": 'USA',
    "Barents Sea": 'Russian Federation',
    "Gran Canaria": 'USA'
}

In [ ]:
space_cor['Country'] = space_cor['Location'].apply(lambda x: extract_country_name(x))
space_cor['Country'] = space_cor['Country'].replace(countries_dict)

In [ ]:
space_cor[['Launcher', 'Navette']] = space_cor['Detail'].str.split('|', 1, expand=True)

In [ ]:
space_cor.columns

Index(['Company Name', 'Location', 'Detail', 'Status Rocket', ' Rocket',
       'Status Mission', 'Year', 'Month', 'Country', 'Launcher', 'Navette'],
      dtype='object')

In [ ]:
space_cor.columns = ['Company Name','Location','Detail', 'Status Rocket', 'Rocket', 'Status Mission','Year','Month','Country', 'Launcher', 'Shuttle']

In [ ]:
space_cor["Rocket"]=space_cor["Rocket"].str.replace(",","")

In [ ]:
space_cor["Rocket"]=pd.to_numeric(space_cor["Rocket"])

In [ ]:
space_cor.drop(["Location","Detail"], axis=1, inplace=True)

In [ ]:
space_cor=space_cor.reindex(columns = ["Company Name","Country","Launcher","Shuttle","Rocket","Status Rocket","Year","Month","Status Mission"])

In [ ]:
space_cor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company Name    4324 non-null   object 
 1   Country         4324 non-null   object 
 2   Launcher        4324 non-null   object 
 3   Shuttle         4324 non-null   object 
 4   Rocket          964 non-null    float64
 5   Status Rocket   4324 non-null   object 
 6   Year            4324 non-null   int64  
 7   Month           4324 non-null   int64  
 8   Status Mission  4324 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 304.2+ KB


#Etude "Country"

In [ ]:
space_cor["Launcher"][0][-1]

' '

In [ ]:
space_cor.head()

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
0,SpaceX,USA,Falcon 9 Block 5,Starlink V1 L9 & BlackSky,50.00,StatusActive,2020,8,Success
1,CASC,China,Long March 2D,Gaofen-9 04 & Q-SAT,29.75,StatusActive,2020,8,Success
2,SpaceX,USA,Starship Prototype,150 Meter Hop,NaN,StatusActive,2020,8,Success
3,Roscosmos,Kazakhstan,Proton-M/Briz-M,Ekspress-80 & Ekspress-103,65.00,StatusActive,2020,7,Success
4,ULA,USA,Atlas V 541,Perseverance,145.00,StatusActive,2020,7,Success


In [ ]:
space_cor.loc[(space_cor["Country"]=="Kenya")]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
1952,ASI,Kenya,Scout G1,San Marco 5,NaN,StatusRetired,1988,3,Success
2936,ASI,Kenya,Scout F1,Explorer 53,NaN,StatusRetired,1975,5,Success
2998,ASI,Kenya,Scout B1,Ariel 5,NaN,StatusRetired,1974,10,Success
3061,ASI,Kenya,Scout D1,San Marco 4,NaN,StatusRetired,1974,2,Success
3187,ASI,Kenya,Scout D1,Explorer 48,NaN,StatusRetired,1972,11,Success
3298,ASI,Kenya,Scout B,Explorer 45,NaN,StatusRetired,1971,11,Success
3357,ASI,Kenya,Scout B,San Marco 3,NaN,StatusRetired,1971,4,Success
3400,ASI,Kenya,Scout B,Explorer 42,NaN,StatusRetired,1970,12,Success
3780,ASI,Kenya,Scout B,San Marco 2,NaN,StatusRetired,1967,4,Success


##Status des missions pour chaque Pays

In [ ]:
space_cor_pivot_table = space_cor.pivot_table(values="Launcher", index="Country", columns="Status Mission", aggfunc='count')

In [ ]:
space_cor_pivot_table= space_cor_pivot_table.fillna(0)

In [ ]:
space_cor_pivot_table["Total"]=(space_cor_pivot_table["Failure"]+space_cor_pivot_table["Partial Failure"]+space_cor_pivot_table["Prelaunch Failure"]+space_cor_pivot_table["Success"])

In [ ]:
space_cor_pivot_table["Success_rate"]=(space_cor_pivot_table["Success"]/(space_cor_pivot_table["Failure"]+space_cor_pivot_table["Partial Failure"]+space_cor_pivot_table["Prelaunch Failure"]+space_cor_pivot_table["Success"]))*100

In [ ]:
space_cor_pivot_table=space_cor_pivot_table.sort_values("Total",ascending=False)

In [ ]:
space_cor_pivot_table

Status Mission,Failure,Partial Failure,Prelaunch Failure,Success,Total,Success_rate
Country,,,,,,
Russian Federation,63.0,30.0,0.0,1305.0,1398.0,93.347639
USA,129.0,33.0,1.0,1188.0,1351.0,87.934863
Kazakhstan,72.0,20.0,1.0,608.0,701.0,86.733238
France,13.0,5.0,0.0,285.0,303.0,94.059406
China,19.0,6.0,0.0,244.0,269.0,90.706320
Japan,10.0,3.0,0.0,113.0,126.0,89.682540
India,8.0,5.0,0.0,63.0,76.0,82.894737
Pacific Ocean,3.0,0.0,0.0,33.0,36.0,91.666667
Iran,8.0,0.0,1.0,5.0,14.0,35.714286


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(space_cor['Status Mission'])
colors = {0 : 'red', 1 : 'Orange', 2 : 'Yellow', 3 : 'Green'}

In [ ]:
CMS = fig = make_subplots(rows = 4 ,cols = 4,subplot_titles=space_cor['Country'].unique())

for i, country in enumerate(space_cor['Country'].unique()):

    counts = space_cor[space_cor['Country'] == country]['Status Mission'].value_counts(normalize = True) * 100
    color = [colors[x] for x in encoder.transform(counts.index)]
    trace = go.Bar(x = counts.index, y = counts.values, name = country,showlegend=False,marker={'color' : color})
    fig.add_trace(trace, row = (i//4)+1, col = (i%4)+1)

fig.update_layout(margin=dict(l=80, r=80, t=100, b=10), title = { 'text' : '<b>Status des missions pour chaque Pays</b>', 'x' : 0.5}, font_family = 'Fira Code',title_font_color= '#cacaca',height = 1000,width = 1100,template = 'plotly_dark')

for i in range(1,5):
    fig.update_yaxes(title_text = 'Percentage',row = i, col = 1)
fig.show()

In [ ]:


CcMS = fig= px.sunburst(space_cor,path = ['Status Mission','Country'] ,color= 'Status Mission', color_discrete_map={"Success":"green", "Failure":"red" , "Partial Failure":'orange' , "Prelauch Failure": 'yellow'}, template = 'plotly_dark' , width=1000, height=1000)

fig.update_layout(margin=dict(l=80, r=80, t=80, b=100), title = { 'text' : '<b>Company and Mission Status</b>', 'x' : 0.5},   uniformtext=dict(minsize=12, mode='hide'))
fig.show()

## Nombre de compagnie par pays

In [ ]:
compag_pivot_table = space_cor.pivot_table(values="Company Name", index="Country", aggfunc=lambda x: len(x.unique())).sort_values("Company Name",ascending=False)

In [ ]:
compag_pivot_table.columns = ["Amount of company"]

In [ ]:
compag_pivot_table.reset_index(inplace=True)

In [ ]:
df = px.data.tips()
CQC = fig = px.bar(compag_pivot_table, x="Country" , y='Amount of company',color="Country", color_discrete_sequence=px.colors.qualitative.Plotly , title="Quantité de compagnie par Pays" , template = 'plotly_dark', labels={"x":"Country"})
fig.show()

##Cout par pays

In [ ]:
cout_pivot_table = space_cor.pivot_table(values="Rocket", index="Country", aggfunc="sum").sort_values("Rocket",ascending=False)

In [ ]:
cout_pivot_table.columns = ["Cost"]

In [ ]:
cout_pivot_table

,Cost
Country,
USA,105287.32
France,16285.00
Kazakhstan,12150.50
China,6368.56
Japan,3700.50
Russian Federation,2189.30
India,2177.00
New Zealand,97.50
Australia,0.00


In [ ]:
df = px.data.tips()
DPP = fig = px.pie(cout_pivot_table[:8], values='Cost', names=(cout_pivot_table[:8].index),template = 'plotly_dark', hover_data=[(cout_pivot_table[:8].index)])
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.update_layout(margin=dict(l=20, r=20, t=40, b=10), title = { 'text' : '<b>Dépense des pays pour les projets spatiaux (en M de $)</b>', 'x' : 0.5}, font_family = 'Fira Code',title_font_color= '#cacaca',height = 700,width = 700,template = 'plotly_dark')
fig.show()

#Etude "Company Name"

##Status des missions pour chaque compagnies

In [ ]:
ECMS = fig = make_subplots(rows = 8 ,cols = 8,subplot_titles=space_cor["Company Name"].unique())

for i, country in enumerate(space_cor["Company Name"].unique()):

    counts = space_cor[space_cor["Company Name"] == country]['Status Mission'].value_counts(normalize = True) * 100
    color = [colors[x] for x in encoder.transform(counts.index)]
    trace = go.Bar(x = counts.index, y = counts.values, name = country,showlegend=False,marker={'color' : color})
    fig.add_trace(trace, row = (i//8)+1, col = (i%8)+1)

fig.update_layout(margin=dict(l=80, r=80, t=80, b=10), title = { 'text' : '<b>Status des missions pour chaque compagnies</b>', 'x' : 0.5}, font_family = 'Fira Code',title_font_color= '#cacaca',height = 2000,width = 1500,template = 'plotly_dark')

for i in range(1,5):
    fig.update_yaxes(title_text = 'Percentage',row = i, col = 1)
fig.show()

In [ ]:
space_cor.loc[(space_cor["Company Name"].str.contains("Douglas"))]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
1894,Douglas,USA,Delta 3920-8,Delta Star (USA 36),NaN,StatusRetired,1989,3,Success


In [ ]:
print(px.colors.qualitative.Set3)

['rgb(141,211,199)', 'rgb(255,255,179)', 'rgb(190,186,218)', 'rgb(251,128,114)', 'rgb(128,177,211)', 'rgb(253,180,98)', 'rgb(179,222,105)', 'rgb(252,205,229)', 'rgb(217,217,217)', 'rgb(188,128,189)', 'rgb(204,235,197)', 'rgb(255,237,111)']


In [ ]:
 CCMS = fig = px.bar(space_cor, x="Company Name", y="Status Mission", color="Status Mission", title="Long-Form Input", color_discrete_map={"Success":"green", "Failure":"red" , "Partial Failure":'rgb(200,120,98)' , "Prelauch Failure": 'rgb(255,255,255)'})
      
fig.update_layout(barmode='stack',bargap=0.3, bargroupgap =0.1,margin=dict(l=80, r=80, t=80, b=10), title = { 'text' : '<b>Status des missions pour chaque compagnies</b>', 'x' : 0.5}, font_family = 'Fira Code',title_font_color= '#cacaca',height = 800,width = 2000,template = 'plotly_dark')
fig.show()

In [ ]:
cout_comp_pivot_table = space_cor.pivot_table(values="Rocket", index="Company Name", aggfunc="sum").sort_values("Rocket",ascending=True)

In [ ]:
cout_comp_pivot_table.reset_index(inplace=True)

In [ ]:
cout_comp_pivot_table

,Company Name,Rocket
0,AEB,0.00
1,ISAS,0.00
2,KARI,0.00
3,KCST,0.00
4,Yuzhmash,0.00
5,Land Launch,0.00
6,Landspace,0.00
7,MITT,0.00
8,OKB-586,0.00
9,OneSpace,0.00


In [ ]:
TCP = fig = px.bar(cout_comp_pivot_table[46:56], y="Company Name", x="Rocket", orientation="h" , color="Rocket" ,color_discrete_sequence=px.colors.sequential.Plasma_r , labels={"Rocket":"Amount"} )
      
fig.update_layout(title = { 'text' : '<b>Les 10 compagnies les plus dépensiaires (en M $)</b>', 'x' : 0.5},template = 'plotly_dark')

fig.show()

In [ ]:
space_cor.loc[(space_cor["Company Name"].str.contains("Ariane"))]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
40,Arianespace,Kazakhstan,Soyuz 2.1b/Fregat,OneWeb #3,48.5,StatusActive,2020,3,Success
48,Arianespace,France,Ariane 5 ECA,JCSAT-17 & GEO-KOMPSAT 2B,200.0,StatusActive,2020,2,Success
54,Arianespace,Kazakhstan,Soyuz 2.1b/Fregat,OneWeb #2,48.5,StatusActive,2020,2,Success
58,Arianespace,France,Ariane 5 ECA,Eutelsat Konnect BB4A & GSAT-30,200.0,StatusActive,2020,1,Success
68,Arianespace,France,Soyuz ST-A/Fregat-M,"CSG-1, CHEOPS & Others",NaN,StatusActive,2019,12,Success
...,...,...,...,...,...,...,...,...,...
2135,Arianespace,France,Ariane 3,"GStar-1, Telecom 1B",NaN,StatusRetired,1985,5,Success
2148,Arianespace,France,Ariane 3,"Arabsat-1A, Brasilsat-A1",NaN,StatusRetired,1985,2,Success
2167,Arianespace,France,Ariane 3,"Spacenet 2, MARECS-2",NaN,StatusRetired,1984,11,Success
2187,Arianespace,France,Ariane 3,"Eutelsat-1 F2, Telecom 1A",NaN,StatusRetired,1984,8,Success


In [ ]:
import plotly.io as pio
list(pio.templates)

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

#Etude "Launcher"

In [ ]:
space_cor_launcher= space_cor.pivot_table(values="Rocket", index=["Launcher","Country"], aggfunc="sum")

In [ ]:
space_cor_launcher.reset_index(inplace=True)

In [ ]:
space_cor_launcher=space_cor_launcher.sort_values("Rocket",ascending=False)

In [ ]:
space_cor_launcher

,Launcher,Country,Rocket
301,Space Shuttle Discovery,USA,17550.0
255,Saturn V,USA,15080.0
18,Ariane 5 ECA,France,15000.0
298,Space Shuttle Atlantis,USA,14850.0
300,Space Shuttle Columbia,USA,12600.0
...,...,...,...
142,H-I UM-129A (9SO),Japan,0.0
141,H-I UM-129A (6SO),Japan,0.0
140,H-I (9 SO),Japan,0.0
136,Feng Bao 1,China,0.0


In [ ]:
TMC = fig = px.bar(space_cor_launcher[:10], y="Launcher", x="Rocket", orientation="h" , color="Country" , labels={"Rocket":"Amount"} )
      
fig.update_layout(title = { 'text' : '<b>Les 10 missions les plus couteuses (en M $)</b>', 'x' : 0.5},template = 'plotly_dark')

fig.show()

#Etude "Year" & "Month"

In [ ]:
LA = fig = px.bar(space_cor["Year"].value_counts(), color=space_cor["Year"].value_counts(), color_continuous_scale=px.colors.sequential.Plasma, labels={'value':'Launches', 'index':'Year','color':'None'})

fig.update_layout(template="plotly_dark",title_text="Lancement par années")
fig.show()

In [ ]:
space_cor_failed=space_cor[(space_cor["Status Mission"].str.contains("Failure"))]

In [ ]:
space_cor_success=space_cor[(space_cor["Status Mission"].str.contains("Success"))]

In [ ]:
space_cor_failed = space_cor_failed.pivot_table(values="Launcher", index="Year", aggfunc="count")

In [ ]:
space_cor_failed.reset_index(inplace=True)

In [ ]:
space_cor_success = space_cor_success.pivot_table(values="Launcher", index="Year", aggfunc="count")

In [ ]:
space_cor_success.reset_index(inplace=True)

In [ ]:
SFA = fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=space_cor_failed["Year"], y=space_cor_failed["Launcher"],
                    mode='lines+markers',
                    name='Failure'))
fig.add_trace(go.Scatter(x=space_cor_success["Year"], y=space_cor_success["Launcher"],
                    mode='lines+markers',
                    name='Success'))

fig.update_layout(template="plotly_dark",title_text="Success & Failure par années")

fig.show()

In [ ]:
space_cor_2018=space_cor[(space_cor["Year"]==2018) & (space_cor["Status Mission"]=="Success")]
space_cor_1977=space_cor[(space_cor["Year"]==1977) & (space_cor["Status Mission"]=="Success")]

In [ ]:
space_cor_2018=space_cor_2018.pivot_table(values="Launcher", index="Country", aggfunc="count")

In [ ]:
space_cor_2018.reset_index(inplace=True)

In [ ]:
space_cor_2018=space_cor_2018.sort_values("Launcher",ascending=False)

In [ ]:
space_cor_2018

,Country,Launcher
0,China,38
7,USA,33
1,France,10
4,Kazakhstan,8
6,Russian Federation,8
2,India,7
3,Japan,6
5,New Zealand,3


In [ ]:
df = px.data.tips()
MRP = fig = px.bar(space_cor_2018, x="Country" , y='Launcher',color="Country", color_discrete_sequence=px.colors.qualitative.Plotly , title="Mission réussi en 2018 par pays" , template = 'plotly_dark', labels={"x":"Country"})
fig.show()

In [ ]:
space_cor_1977=space_cor_1977.pivot_table(values="Launcher", index="Country", aggfunc="count")
space_cor_1977.reset_index(inplace=True)
space_cor_1977=space_cor_1977.sort_values("Launcher",ascending=False)

In [ ]:
df = px.data.tips()
LAO = fig = px.bar(space_cor_1977, x="Country" , y='Launcher',color="Country", color_discrete_sequence=px.colors.qualitative.Plotly , title="Mission réussi en 1977 par pays" , template = 'plotly_dark', labels={"x":"Country"})
fig.show()

In [ ]:
space_cor_2020=space_cor[(space_cor["Year"]==2020)]

In [ ]:
fig = px.bar(space_cor_2020, x="Country" , y='Launcher',color="Status Mission", color_discrete_sequence=px.colors.qualitative.Plotly , title="Mission en 2020 par pays" , template = 'plotly_dark', labels={"x":"Country"})
fig.show()

#Etude "Status Mission"

In [ ]:
space_cor["Status Mission"].unique()

array(['Success', 'Failure', 'Prelaunch Failure', 'Partial Failure'],
      dtype=object)

In [ ]:
space_cor.loc[(space_cor["Status Mission"]=="Prelaunch Failure")]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
106,ISA,Iran,Safir-1B+,Nahid-1,NaN,StatusActive,2019,8,Prelaunch Failure
413,SpaceX,USA,Falcon 9 Block 3,AMOS-6,62.0,StatusRetired,2016,9,Prelaunch Failure
1032,AEB,Brazil,VLS-1,"SATEC, UNOSAT",NaN,StatusActive,2003,8,Prelaunch Failure
3816,RVSN USSR,Kazakhstan,Soyuz,Soyuz 7K-OK n†­1,NaN,StatusRetired,1966,12,Prelaunch Failure


In [ ]:
space_cor.loc[(space_cor["Status Mission"]=="Failure") & (space_cor["Year"]==1986)]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
2040,RVSN USSR,Russian Federation,Tsyklon-3,Cosmos 1786 to 1791,NaN,StatusRetired,1986,10,Failure
2061,Arianespace,France,Ariane 2,Intelsat-5A 14,NaN,StatusRetired,1986,5,Failure
2066,Martin Marietta,USA,Titan 34D,KH-9,NaN,StatusRetired,1986,4,Failure
2079,NASA,USA,Space Shuttle Challenger,STS-51-L,450.0,StatusRetired,1986,1,Failure


In [ ]:
space_cor.loc[(space_cor["Status Mission"]=="Partial Failure")]

,Company Name,Country,Launcher,Shuttle,Rocket,Status Rocket,Year,Month,Status Mission
127,Exos,USA,SARGE,Launch 3,NaN,StatusActive,2019,6,Partial Failure
157,Exos,USA,SARGE,Launch 2,NaN,StatusActive,2019,3,Partial Failure
219,Exos,USA,SARGE,Launch 1,NaN,StatusActive,2018,8,Partial Failure
277,Arianespace,France,Ariane 5 ECA,SES-14/GOLD & Al Yah-3,200.0,StatusActive,2018,1,Partial Failure
336,Roscosmos,Kazakhstan,Soyuz 2.1a/Fregat,Kanopus-V IK & Rideshares,48.5,StatusActive,2017,7,Partial Failure
...,...,...,...,...,...,...,...,...,...
4277,US Navy,USA,Vanguard,Vanguard 3,NaN,StatusRetired,1959,9,Partial Failure
4280,NASA,USA,Atlas-D Mercury,Big Joe 1,NaN,StatusRetired,1959,9,Partial Failure
4292,RVSN USSR,Kazakhstan,Vostok,Luna-1,NaN,StatusRetired,1959,1,Partial Failure
4294,AMBA,USA,Juno II,Pioneer 3,NaN,StatusRetired,1958,12,Partial Failure


In [ ]:
!pip install datapane 

In [ ]:
import datapane as dp

In [ ]:
your_token = '4ae0d73560f248d240c71910b9beac09a86828ea'
dp.login(token=your_token)

Connected successfully to https://datapane.com as ramdanetlemcani


'ramdanetlemcani'

In [ ]:
report=dp.Report("Analysis of space missions",
    dp.Group(
        dp.Table(cout_pivot_table, caption=f'Dépense par Pays'),
        dp.Table(space_cor_2018, caption=f'Lanceur par Pays'),        
        columns=2,label = 'Dépense des Pays'),
    dp.Group(
        dp.Plot(CMS, caption=f'Missions Status per Counrty'),
        dp.Plot(CcMS, caption=f'Company and Mission Status'),
        columns=2),
    dp.Group(
        dp.Plot(CQC, caption=f"Company quantity/Country"),
        columns=1, label='Number of Compnay per Country'),
    dp.Group(
        dp.Plot(ECMS, caption=f'Status des missions pour chaque compagnies'),
        columns=1),
    dp.Group(
        dp.Plot(CCMS, caption=f'Status des missions pour chaque compagnies'),
        columns=1),
    dp.Group(
        dp.Plot(TCP, caption=f'10 Compagnies les plus dépensieres'),
        dp.Plot(DPP, caption=f'Dépense des pays pour les projets spatiaux (en M de $)'),
        columns=2),
    dp.Group(
        dp.Plot(TMC, caption=f'10 Lanceur les plus dépensieres'),
        columns=1),
    dp.Group(
        dp.Plot(LA, caption=f'Taux de réussite VS Echec'),
        columns=1),
    dp.Group(
        dp.Plot(LAO, caption=f'Taux de réussite VS Echec'),
        dp.Plot(MRP, caption=f'Taux de réussite VS Echec'),
        columns=2),
    dp.Group(
        dp.Plot(SFA, caption=f'Taux de réussite VS Echec'),
        columns=1),
        
    type=dp.ReportType.DASHBOARD)

In [ ]:
report.publish(name = "Etudes des missions spatiales depuis 1957")

Publishing document and associated data - *please wait...*
Report successfully published at https://datapane.com/u/ramdanetlemcani/reports/test/ - you can edit and add additional text from the link
